# Scraping przepisów z www.przepisy.pl



Jest API:

Wylistowanie z kategorii
https://api2.przepisy.pl/recipes/categories?type=categories&category=/przepisy/ciasta&flat=false

wylistowanie z podkategorii
https://api2.przepisy.pl/recipes/search/?page=1&per_page=32&category=babka-piaskowa&sort=popular

Brak api na skladniki


In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [37]:
root_url = 'https://www.przepisy.pl/przepis/'

In [38]:
cakes = [
    'ciasto-marmurkowe',
    'ciasto-cytrynowe-7860',
    'babka-pomaranczowa-11999',
    'babka-ucierana-2832'
]

In [39]:
def get_page(url):
    return BeautifulSoup(requests.get(url).text, 'lxml')

In [40]:
def get_ingredients_ul(url):
    bs = get_page(url)
    return bs.select('ul.ingredient-ul li.ingredient-li')

In [41]:
def get_ingredients(url):
    ingredients_ul = get_ingredients_ul(url)
    ingredients = []

    for ingredient_li in ingredients_ul:
        ingredient_name = ingredient_li.select_one('div.ingredient-name input').get('id')
        ingredient_quantity_tag = ingredient_li.select_one('span.quantity')
        if ingredient_quantity_tag:
            ingredient_quantity = ingredient_quantity_tag.text
        else: 
            ingredient_quantity = '---'
    
        ingredients.append( [url, ingredient_name, ingredient_quantity] )
    return ingredients

In [42]:
ingredients_df = pd.DataFrame(columns=['URL','nazwa_skladnika','ilosc'])

In [43]:
urls_to_cakes = [ root_url + cake for cake in cakes ]

In [50]:
ingredients = []
for url_to_cake in urls_to_cakes:
    print(f"Requesting {url_to_cake}")
    ingredients = ingredients +  get_ingredients(url_to_cake)
    time.sleep( 15 )
    

Requesting https://www.przepisy.pl/przepis/ciasto-marmurkowe
Requesting https://www.przepisy.pl/przepis/ciasto-cytrynowe-7860
Requesting https://www.przepisy.pl/przepis/babka-pomaranczowa-11999
Requesting https://www.przepisy.pl/przepis/babka-ucierana-2832


In [52]:
ingredients

[['https://www.przepisy.pl/przepis/ciasto-marmurkowe',
  'Kostka do pieczenia Kasia',
  '1 szt.'],
 ['https://www.przepisy.pl/przepis/ciasto-marmurkowe',
  'cukier puder',
  '1 szklanka'],
 ['https://www.przepisy.pl/przepis/ciasto-marmurkowe',
  'cukier waniliowy',
  '1 opak.'],
 ['https://www.przepisy.pl/przepis/ciasto-marmurkowe', 'jajko', '4 szt.'],
 ['https://www.przepisy.pl/przepis/ciasto-marmurkowe',
  'proszek do pieczenia',
  '3 łyżeczki'],
 ['https://www.przepisy.pl/przepis/ciasto-marmurkowe', 'mąka', '2 szklanki'],
 ['https://www.przepisy.pl/przepis/ciasto-marmurkowe',
  'mleko',
  '0,5 szklanki'],
 ['https://www.przepisy.pl/przepis/ciasto-marmurkowe', 'kakao', '1 łyżka'],
 ['https://www.przepisy.pl/przepis/ciasto-marmurkowe',
  'cukier puder do dekoracji ',
  '3 dag'],
 ['https://www.przepisy.pl/przepis/ciasto-cytrynowe-7860',
  'Kostka do pieczenia Kasia',
  '1 szt.'],
 ['https://www.przepisy.pl/przepis/ciasto-cytrynowe-7860',
  'cukier puder',
  '1 szklanka'],
 ['https://w

In [53]:
ingredients_df = pd.DataFrame(ingredients, columns=['URL','nazwa_skladnika','ilosc'])

In [55]:
#ingredients_df =  ingredients_df.append(ingredients)
#ingredients_df

In [56]:
ingredients_df.to_csv('./data/ingredients_cakes.csv', index=False, header=True)